In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from scipy import ndimage
import skvideo.io  
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pdb
import pandas as pd
import datetime
import scipy as sp
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.nn.functional as F
from matplotlib.patches import Ellipse
from skimage.transform import rescale, resize
from skimage import io
import matplotlib.image as mpimg
import imageio
%matplotlib inline
#plt.style.use(['dark_background'])
#plt.style.use('ggplot')

In [3]:
VIDEOPATH = '/notebooks/SkipPrediction/notebooks/brainio/video2_ds.mp4' 
EEGPATH =   '/notebooks/SkipPrediction/notebooks/brainio/eeg2_1.csv'
BIASIMAGEPATH = '/notebooks/SkipPrediction/notebooks/brainio/illusion.jpg'
MODELPATH = ''
FRAMERATE = 30

In [4]:
def load_video(path_to_video, down_sample_rate):
    vid = imageio.get_reader(path_to_video,  'ffmpeg')
    h, w, c = vid.get_data(0).shape
    dframes = int(np.ceil(vid.count_frames()/down_sample_rate))
    pix = np.zeros((h*w*dframes,c))
    for idx, frame in enumerate(np.arange(0, vid.count_frames(), step = down_sample_rate)):#, step = down_sample_rate)):
        pix[h*w*idx:h*w*(idx+1),:] = vid.get_data(frame).reshape(h*w, c)
    return pix, h, w, int(vid.count_frames()/down_sample_rate)
    
def load_bias(path_to_bias, h, w):
    biasimage = io.imread(path_to_bias)
    #biasimage = rescale(biasimage, 0.25, anti_aliasing=False)
    biasimage = resize(biasimage, (h, w), anti_aliasing=True)
    
    return biasimage.reshape(h*w, 3)
    
def load_eeg(path_to_eeg, down_sample_rate = 10):
    eegdata = pd.read_csv(path_to_eeg, sep = ' ', header=None)
    eegdata = eegdata.iloc[::down_sample_rate, :]
    return eegdata
    
def generate_gradient(h, w, axisrange=1):
    return(np.meshgrid(np.linspace(-axisrange, axisrange, num=h),np.linspace(-axisrange, axisrange, num=w))[0])
    
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path_to_video, path_to_eeg, path_to_bias, down_sample_rate=10, transform = None):
        
        self.video, h, w, num_video_frames = load_video(path_to_video, down_sample_rate)
        self.eeg = load_eeg(path_to_eeg, down_sample_rate)
        num_eeg_frames, _ = self.eeg.shape
        if (num_eeg_frames == num_video_frames):
             num_ref_frames = num_eeg_frames 
        else:
            num_ref_frames = np.min([num_eeg_frames, num_video_frames])  
        self.eeg = self.eeg.loc[self.eeg.index.repeat(h*w)].to_numpy() 
        xx, yy = generate_gradient(h, w), generate_gradient(w, h, axisrange=w/h).transpose()
        zz = np.sqrt(xx**2 + yy**2)
        gframes = np.repeat(np.concatenate((xx[:,:,np.newaxis], yy[:,:,np.newaxis], zz[:,:,np.newaxis]), axis=2)
                            .reshape(h*w, 3), 
                            num_ref_frames, axis=0)
        eframes = self.eeg[:num_ref_frames*h*w, :]
        vframes = self.video[:num_ref_frames*h*w, :]
        gframes = gframes[:num_ref_frames*h*w, :]
        self.data = np.concatenate((gframes, vframes, eframes),axis=1)
        self.bias = np.repeat(load_bias(path_to_bias, h, w), int(np.ceil(num_ref_frames)),axis=0)
        self.transform = transform
 
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data[index,:]
        label = self.bias[index,:]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [5]:
train_dataset = Dataset( VIDEOPATH, EEGPATH, BIASIMAGEPATH, transform=None)

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
train_iter = iter(train_loader)
images, labels = train_iter.next()

print('images shape on batch size = {}'.format(images.size()))
print('labels shape on batch size = {}'.format(labels.size()))

images shape on batch size = torch.Size([64, 22])
labels shape on batch size = torch.Size([64, 3])


In [21]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=22, out_features=128)
        #self.fc1_bn = nn.BatchNorm1d(num_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        #self.fc2_bn = nn.BatchNorm1d(num_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=32)
        #self.fc3_bn = nn.BatchNorm1d(num_features=32)
        self.fc4 = nn.Linear(in_features = 32, out_features = 3)
    
        
    def forward(self,x):
        x = torch.tanh(self.fc1(x)) #self.fc1_bn(self.fc1(x)))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        
        return(x)
    
    def regularizer(self, reg):
        return (self.fc1.weight.pow(2).mean() + self.fc2.weight.pow(2).mean() 
                + self.fc3.weight.pow(2).mean() + self.fc4.weight.pow(2).mean())*reg
    

In [22]:
net = Net().cuda()
outputs = net(images.cuda().float()).squeeze()
print('Lets see the outputs for the random net', outputs)

Lets see the outputs for the random net tensor([[0.4714, 0.4811, 0.4305],
        [0.4576, 0.4686, 0.4803],
        [0.4605, 0.4699, 0.4514],
        [0.4613, 0.4677, 0.4663],
        [0.4621, 0.4753, 0.4582],
        [0.4731, 0.4567, 0.4910],
        [0.4607, 0.4703, 0.4810],
        [0.4600, 0.4714, 0.4589],
        [0.4572, 0.4680, 0.4964],
        [0.4345, 0.4942, 0.4891],
        [0.4568, 0.4627, 0.4713],
        [0.4466, 0.4810, 0.5308],
        [0.4592, 0.5102, 0.4314],
        [0.4618, 0.4647, 0.4678],
        [0.4776, 0.4551, 0.4948],
        [0.4452, 0.4871, 0.5294],
        [0.4661, 0.4799, 0.4392],
        [0.4705, 0.5006, 0.4269],
        [0.4566, 0.4706, 0.4853],
        [0.4615, 0.4759, 0.4806],
        [0.4576, 0.4643, 0.4730],
        [0.4698, 0.5017, 0.4240],
        [0.4614, 0.5102, 0.4291],
        [0.4672, 0.5039, 0.4232],
        [0.4697, 0.4975, 0.4287],
        [0.4613, 0.4694, 0.4770],
        [0.4599, 0.4761, 0.4830],
        [0.4570, 0.4802, 0.4849],
        

In [23]:
import torch.optim as optim

reg = 0
LR = 0.00001
#aplacel2 = LaplaceL2() 
criterion = nn.MSELoss() #nn.PoissonNLLLoss() 
optimizer = optim.Adam(net.parameters(),lr=LR)

In [24]:
NUM_EPOCH = 1

for epoch in range(NUM_EPOCH):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the input
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.cuda().float()).squeeze()
        labels = labels.float()
        #loss = criterion(outputs, labels)  #labels.cuda())
        loss = criterion(outputs, labels.cuda()) + net.regularizer(reg)
        
        loss.backward(retain_graph=True)
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0

print('Finished Training')

[1,  1000] loss: 0.03185
[1,  2000] loss: 0.02684
[1,  3000] loss: 0.02686
[1,  4000] loss: 0.02665
[1,  5000] loss: 0.02667
[1,  6000] loss: 0.02672
[1,  7000] loss: 0.02636
[1,  8000] loss: 0.02644
[1,  9000] loss: 0.02642
[1, 10000] loss: 0.02630
[1, 11000] loss: 0.02647
[1, 12000] loss: 0.02624
[1, 13000] loss: 0.02640
[1, 14000] loss: 0.02624
[1, 15000] loss: 0.02610
[1, 16000] loss: 0.02625
[1, 17000] loss: 0.02577
[1, 18000] loss: 0.02569
[1, 19000] loss: 0.02594
[1, 20000] loss: 0.02578
[1, 21000] loss: 0.02587
[1, 22000] loss: 0.02586
[1, 23000] loss: 0.02582
[1, 24000] loss: 0.02550
[1, 25000] loss: 0.02543
[1, 26000] loss: 0.02573
[1, 27000] loss: 0.02540
[1, 28000] loss: 0.02569
[1, 29000] loss: 0.02561
[1, 30000] loss: 0.02553
[1, 31000] loss: 0.02541
[1, 32000] loss: 0.02554
[1, 33000] loss: 0.02551
[1, 34000] loss: 0.02548
[1, 35000] loss: 0.02528
[1, 36000] loss: 0.02540
[1, 37000] loss: 0.02519
[1, 38000] loss: 0.02511
[1, 39000] loss: 0.02529
[1, 40000] loss: 0.02531


In [25]:
# PARAMETERS
x_layer = 0
y_layer = 1
r_layer = 2
num_eeg_layers = 8
eeg_layers = range(r_layer, r_layer+num_eeg_layers)

# ALLOCATION
grouped = []

for idx, layer in enumerate(net.named_parameters()):
    if idx==0:
        weights = layer[1].detach().cpu().numpy().transpose()
        grouped.append({'weights': weights[r_layer+4:,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[x_layer, :][np.newaxis,:], 
                       'bias': np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[y_layer, :][np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[r_layer,:][np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        for i in np.arange(3):
            grouped.append({'weights': weights[r_layer+i, :][np.newaxis,:], 
                           'bias':  np.array(0),
                           'activation': 'tanh'})  
        grouped.append({'weights': np.zeros((128))[np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
    elif idx==1:
        continue
    elif idx%2==0:
        weights = layer[1].detach().cpu().numpy().transpose()
    else:
        # extract biases
        bias = layer[1].detach().cpu().numpy().transpose()
        grouped.append({'weights': weights, 
                       'bias': bias,
                       'activation': 'tanh'})
        

In [26]:
[g['weights'] for g in grouped]

[array([[ 0.13419105,  0.10357777,  0.09121027, ..., -0.11948179,
         -0.14664254, -0.09729779],
        [ 0.02213232,  0.03351563, -0.04989863, ...,  0.07303265,
          0.21928878,  0.18077382],
        [ 0.01961983,  0.0156246 ,  0.07296897, ..., -0.05238726,
          0.11031671, -0.23523319],
        ...,
        [ 0.02425811,  0.01026146, -0.05588618, ..., -0.00031853,
          0.09709542, -0.02615653],
        [-0.03386963, -0.1708096 , -0.0158639 , ...,  0.11939831,
          0.2771896 ,  0.02156863],
        [ 0.11026907,  0.06282111, -0.18265785, ..., -0.05967565,
          0.07297625, -0.11889093]], dtype=float32),
 array([[-9.32326391e-02,  7.17796683e-02,  1.87051475e-01,
          2.58743942e-01, -8.22711363e-02, -4.07451317e-02,
         -2.14901239e-01, -2.18394637e-01, -9.62023586e-02,
         -1.99569330e-01,  6.86448812e-02,  5.59678338e-02,
         -2.52464954e-02,  8.37474689e-02, -3.95889208e-02,
         -5.86593710e-02,  1.78345591e-01, -1.02875866e-01

In [27]:
np.save('trained_weights_video2_reg0', grouped, allow_pickle=True, fix_imports=False)

### Archive

In [ ]:
def load_video(path_to_video, down_sample_rate):
    vid = imageio.get_reader(path_to_video,  'ffmpeg')
    h, w, c = vid.get_data(0).shape
    dframes = int(np.ceil(vid.count_frames()/down_sample_rate))
    pix = np.zeros((h*w*dframes,c))
    for idx, frame in enumerate(np.arange(0, vid.count_frames(), step = down_sample_rate)):#, step = down_sample_rate)):
        pix[h*w*idx:h*w*(idx+1),:] = vid.get_data(frame).reshape(h*w, c)
    #vid = skvideo.io.vread(path_to_video) 
    return pix, h, w, vid.count_frames()
    
def load_bias(path_to_bias, h, w):
    biasimage = io.imread(path_to_bias)
    #h, w, c = 480, 640, 3
    #biasimage = rescale(biasimage, 0.25, anti_aliasing=False)
    biasimage = resize(biasimage, (h, w), anti_aliasing=True)
    
    return biasimage.reshape(h*w, 3)
    
def load_eeg(path_to_eeg, down_sample_rate):
    eegdata = pd.read_csv(path_to_eeg, sep = ' ', header=None)
    eegdata = eegdata.iloc[::down_sample_rate, :]
    return eegdata
    
def generate_gradient(h, w, axisrange=1):
    return(np.meshgrid(np.linspace(-axisrange, axisrange, num=h),np.linspace(-axisrange, axisrange, num=w))[0])
    
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path_to_video, path_to_eeg, path_to_bias, down_sample_rate=10, transform = None):
        
        self.video, h, w, num_video_frames = load_video(path_to_video, down_sample_rate)
        # h, w, color_channels, num_video_frames = self.video.shape
        # self.video = np.reshape(self.video, h*w*num_video_frames, time_axis)
        self.eeg = load_eeg(path_to_eeg, down_sample_rate)
        num_eeg_frames, _ = self.eeg.shape
        if (num_eeg_frames == num_video_frames):
             num_ref_frames = num_eeg_frames 
        else:
            num_ref_frames = np.min([num_eeg_frames, num_video_frames])  
        self.eeg = self.eeg.loc[self.eeg.index.repeat(h*w)].to_numpy() 
        xx, yy = generate_gradient(h, w), generate_gradient(w, h, axisrange=w/h).transpose()
        zz = np.sqrt(xx**2 + yy**2)
        gframes = np.repeat(np.concatenate((xx[:,:,np.newaxis], yy[:,:,np.newaxis], zz[:,:,np.newaxis]), axis=2)
                            .reshape(h*w, 3), 
                            num_ref_frames, axis=0) # int(num_ref_frames/down_sample_rate)
        eframes = self.eeg[:num_ref_frames*h*w, :]
        vframes = self.video[:num_ref_frames*h*w, :]
        gframes = gframes[:num_ref_frames*h*w:down_sample_rate, :]
        print(eframes.shape, vframes.shape, gframes.shape)
        self.data = np.concatenate((gframes, vframes, eframes),axis=1)
        self.bias = np.repeat(load_bias(path_to_bias, h, w), int(np.ceil(num_ref_frames/down_sample_rate)),axis=0)
        self.transform = transform
 
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data[index,:]
        label = self.bias[index,:]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label